In [ ]:

import json
import os

from pathlib import Path
from typing import Dict, Tuple
from unittest.mock import MagicMock, patch

import numpy as np
import pandas as pd
import pytest

from dl import TEST_DIR
from dl.config.label_map_helper import LabelMapHelper
from dl.dataset import cutnpaste_presampling
from dl.dataset.cutnpaste_preparer import CutnPastePreparer
from dl.dataset.dataframe_schema_constants import (
    CAMERA_LOCATION,
    CAMERA_LOCATION_RIGHT,
    HDR_MODE,
    UNIQUE_ID,
)
from dl.utils.config import (
    DEFAULT_CUTNPASTE_PREPARER_PARAMS,
    DEFAULT_CUTNPASTE_RUNTIME_PARAMS,
    DEFAULT_IMAGE_SIZE,
    HUMAN_LABELSTR_IN_ANNOTATIONS,
    MAX_DEPTH,
)
from dl.utils.cutnpaste import CutnPaste
from dl.utils.helpers import load_master_csv, save_master_csv
from tests.example_images.load_image import DatasetWithDatamode

import matplotlib.pyplot as plt
%matplotlib inline


In [ ]:
from pytorch_lightning.utilities.rank_zero import rank_zero_only

In [ ]:
from dl.utils.helpers import load_master_csv
dataset_path = '/data2/jupiter/datasets/Jupiter_train_v6_2/'
df = load_master_csv(dataset_path + 'master_annotations_20231019_clean.csv')
from dl.dataset.datamodes.npz.rgbd import RGBDNPZ
rgbdnpz = RGBDNPZ(dataset_path)
artifacts = rgbdnpz.get_artifacts(df.iloc[0])
plt.imshow(artifacts['depth'])

In [ ]:
# bad_df = df[df['id'] == '5f9d6a1bd61e89229bd3c90a']
# save_master_csv(bad_df, dataset_path + 'one_image_sus_label.csv')
df = load_master_csv(csv_path=dataset_path + 'one_image_sus_label.csv')

In [ ]:
df[df['id'] == '5f9d6a1bd61e89229bd3c90a']['label_map']

In [ ]:
from dl.dataset.datamodes.npz.rgbd import RGBDNPZ

rgbdnpz = RGBDNPZ(dataset_path)

In [ ]:
row = df[df['id'] == '5f9d6a1bd61e89229bd3c90a'].iloc[0]

In [ ]:
artifacts = rgbdnpz.get_artifacts(row)

In [ ]:
import matplotlib as mpl
classlabels_viz_colors = [
    "white",  # Ignored labels mapped to white so that they do not change color when masked - we don't care about them either way, so a color change does not make sense
    "green",
    "yellow",
    "blue",
    "red",
    "magenta",
    "cyan",
    "lightseagreen",
    "brown",
    "magenta",
    "olive",
    "wheat",
    "black",
]
classlabels_viz_bounds = [-1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

classlabels_viz_cmap = mpl.colors.ListedColormap(classlabels_viz_colors)
classlabels_viz_norm = mpl.colors.BoundaryNorm(classlabels_viz_bounds, classlabels_viz_cmap.N)

In [ ]:
np.unique(artifacts['label'])

In [ ]:
np.unique(label)

In [ ]:
label = artifacts['label'].copy()
label[label == 0] = -1
label[label == 2] = 0
label[label == 11] = 5
plt.imshow(label,classlabels_viz_cmap,classlabels_viz_norm,interpolation="nearest",alpha=None)

In [ ]:
plt.imshow(artifacts['image'])

In [ ]:
for i in range(100):
    s = np.sum(rgbdnpz.get_artifacts(df.iloc[i])['label'] == 11)
    if s > 0 and df.iloc[i]['label_map']['11'] == 'Humans':
        print(i, s)

In [ ]:
base_ind = 2
hum_ind = 50
im_no_hum = rgbdnpz.get_artifacts(df.iloc[base_ind])['image']
im_w_hum =  rgbdnpz.get_artifacts(df.iloc[hum_ind])['image']
mask =(rgbdnpz.get_artifacts(df.iloc[hum_ind])['label'] == 11)[:,:,None]

In [ ]:
# Vanilla CNP
# plt.imshow(im_w_hum * mask)
# plt.imshow(im_no_hum)
plt.imshow(im_no_hum * (1 - mask) + im_w_hum * mask)

In [ ]:
from pietorch import blend_dst_numpy
import torch
target : torch.Tensor = torch.Tensor(im_no_hum) # 3 x N x M image to be blended into
source : torch.Tensor = torch.Tensor(im_w_hum) # 3 x H x W image to be blended
human_mask : torch.Tensor = torch.Tensor(mask)[:,:,0] # H x W mask of which pixels from source to be included
corner : torch.Tensor = torch.IntTensor([0,0]) # [y, x] coordinate of location in target for source to be blended
# result = blend(target, source, human_mask,corner, True, channels_dim=2)
result = blend_dst_numpy(im_no_hum, im_w_hum, mask[:,:,0],corner.numpy(), False, channels_dim=2)

plt.imshow(result)
